# Obliczanie zwrotu z inwestycji


W tej części projektu, na podstawie sygnałów generwanych przez poprzedni kod pythona obliczone zostaną dzienne stopy zwrotu z rozważanych strategii

In [7]:
### BIBLIOTEKI ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### ŚCIEŻKI ##
file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/rates_data/rates_data.xlsx"
file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/prices_cleaned.xlsx"
file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/GPW capitalization USD mies.xlsx"
file_path4 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/zestawienie nazw.xlsx"



## Wczytywanie danych


Sygnały są zapisane w 15 strategiach (każda w osobnym arkuszu w pliku excel). Odczytuje wszytskiie do słownika ramek danych


In [8]:
prices = pd.read_excel(file_path2)

prices.iloc[:, 1:] = prices.iloc[:, 1:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))

capitalization = pd.read_excel(file_path3)

accuracy = pd.read_excel(file_path4)


sheets_dict = pd.read_excel(file_path1, sheet_name=None)

sygnals = {} # Pusty słownik do zappisywania 
for sheet_name, dataframe in sheets_dict.items():
    sygnals[sheet_name] = dataframe

# Odwoływanie sie do konkretnych ramek danych -> sygnals['rates_10_1']

C:\Users\olgas\AppData\Local\Temp\ipykernel_22236\322302912.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prices.iloc[:, 1:] = prices.iloc[:, 1:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))


## Dzienne stopy zwrotu 

In [ ]:
def daily_returns(df):
    result = pd.DataFrame(columns=df.columns)  # Nowa ramka danych do przechowywania wyników

    for col in df.columns:
        if col == 'Date':
            
            result['Date'] = df['Date'][1:].reset_index(drop=True)
        else:
            # Oblicz dzienne stopy zwrotu
            rates_values = []  # Pusta lista na dzienne stopy zwrotu
            for i in range(len(df) - 1):
                rate = (df[col].iloc[i + 1] - df[col].iloc[i]) / df[col].iloc[i]
                rates_values.append(rate)  # Dodaj wynik do listy

            result[col] = pd.Series(rates_values).reset_index(drop=True)

    return result

daily_returns = daily_returns(prices)

## Ujednolicenie kolumn 


W badaniu potrzeba ujednolicić kolumny w ramkach danych z kapitalizacją i cenami i sygnałami


In [9]:
columns_to_remove = accuracy.loc[accuracy['czy zgodnosc'].isnull(), 'Profesora'].tolist() # Kolumny których nie ma w mojej tabeli do usuniecia 

capitalizationv2 = capitalization.drop(columns=columns_to_remove) # usunięcie wartości których ja u siebie nie mam 

capitalizationv3 = capitalizationv2[prices.columns] # ustawienie w tej samej kolejności kolumny



## Okres inwestycji - 1 dzień 


W przypadku okresu sprzedaży równego 1 dzień, nie trzeba liczyć średnich dziennych stóp zwrotu z uwzględnieniem tego okresu 


In [ ]:
def one_day(df, price, kap_min, kap_max ):

    result = pd.DataFrame()

    for i in range(len(df)):
        row_sum = 0
        count = 0

        for col in df.columns:
            if col == 'Date':
                result['Date'] = df['Date']
            else:
             
                if df[col].iloc[i].isna() == False:
                    
                    if df[col].iloc[i] > price and kap_min <= df[col].iloc[i] <= kap_max:
                        row_sum += df[col].iloc[i]
                        count += 1
        
        result['Mean'].iloc[i] = row_sum/count
        result['Count'].iloc[i] = count

    return result



    
